In [ ]:

# define widgets - NEED TO DEFINE IT ONCE
# dynamic variables (pass it from ADF)
# first time runtime parameter
# dbutils.widgets.dropdown("environment", "dev", ["dev","uat","prod"])
# dbutils.widgets.dropdown("new_training", "False", ["True","False"])
# dbutils.widgets.dropdown("system_name", "bimodelapi", ["bimodelapi"])
# dbutils.widgets.text("system_name", "","")
# dbutils.widgets.remove("pbiapi")
environment = dbutils.widgets.get("environment")
new_training = dbutils.widgets.get("new_training")
system_name = dbutils.widgets.get("system_name")
if environment in {"prod"}:
    aml_compute_cluster_name = "cc-bi-ml-prod01"
else:
    aml_compute_cluster_name = "cc-bi-ml-devqa01"

print(environment, system_name, new_training, aml_compute_cluster_name)



In [ ]:
%run ./data_load_rev_region_forecast

In [ ]:
# Prepare Data & Set time column and series columns
print("Preparing data")
# set variales

target_column_name = "Revenue"
time_column_name = "End_of_Month"
time_series_id_column_names = ["Relative_Offset", "Sub_Region_Code"]
sort_cols_snp = ["End_of_Month", "Snapshot_Date", "Sub_Region_Code"]
sort_cols_eom = ["End_of_Month", "Sub_Region_Code"]

exclude_sub_region = ["Singapore", "NA"]

pd.set_option("display.float_format", lambda x: "%.2f" % x)

talenthist1 = talenthist.copy()  # .toPandas() to convert from spark df to pandas
pipehist1 = pipehist.copy()
opphist1 = opphist.copy()
projhist1 = projhist.copy()
revhist1 = revhist.copy()
pipetrend1 = pipetrend.copy()
projectactualshist1 = projectactualshist.copy()

# pipetrend1.drop(columns="Snapshot_Date_Short", axis=1, inplace=True)

pipehist1.rename(columns={"Snapshot_Date_Short": "Snapshot_Date"}, inplace=True)
pipetrend1.rename(columns={"Snapshot_Date_Short": "Snapshot_Date"}, inplace=True)
pipetrend1.rename(columns={"Snapshot_End_of_Month": "End_of_Month"}, inplace=True)
#talenthist1["Billable_Headcount"] = (
#    talenthist1["Headcount"] + talenthist1["Headcount_Contingent"]
#)
#talenthist1.drop(columns=["Headcount"], axis=1, inplace=True)

numeric_cols = [
    "Pipeline",
    "Pipeline_at_100_Percent",
    "Yield",
    "Pipeline_Trend",
    "Revenue",
    "Project_Period_Count",
    "Project_Count",
    "Project_Period_Price",
    "Project_Price",	
    "Conversions",
    "Opportunity_Period_Count",
    "Opportunity_Count",
    "Current_Opp._Period_Value",
    'Opportunity Value',
    "Win_Rate",
    "Headcount",
    "Utilization_Billable"
]

df_list = [
    pipehist1,
    talenthist1,
    revhist1,
    opphist1,
    projhist1,
    pipetrend1,
    projectactualshist1
]
for x in df_list:
    x.infer_objects()

    # convert date to to_datetime

    x = convert_date_cols(x)

    # coerce numeric_cols to numeric
    x = coerce_to_numeric(x)

    # replace null values to NA

    x["Sub_Region_Code"] = x["Sub_Region_Code"].replace(np.nan, "NA", regex=True)

    x = x.replace(np.nan, 0, regex=True)

    # make all dates to End of Month values to later merge
    if "End_of_Month" in x.columns:

        x["End_of_Month"] = x["End_of_Month"] + pd.offsets.MonthEnd(0)

    if "Snapshot_Date" in x.columns:

        # x['Snapshot_Date'] = x['Snapshot_Date'] + pd.offsets.MonthEnd(0)

        x.sort_values(by=["End_of_Month", "Snapshot_Date", "Sub_Region_Code"]).reset_index(drop=True)
    else:
        x.sort_values(by=sort_cols_eom).reset_index(drop=True)
    
    print(get_df_name(x), ":", x.shape)
    '''
    display(x.info())
    display(x.tail())
    '''


In [ ]:

# Cross join to get cross-join of all unquie values of End_of_Month + Relative_Offset + Snp_Seq_No + Sub_Region_Code with forward filling the last snapshot date for that relative_month
print("Preparing eom_region_snp_final")
# Set Parameters---------------------------------------------------------------------------------------------------------------------

past_period = 12
n_test_periods = 4
nan_value = 0
dt1 = pd.to_datetime('2018-07-31') #ignore revenues before pipeline data was not there

main_filter = "Relative_Offset >= -@past_period and \
                Relative_Offset < @n_test_periods and \
                Relative_Snapshot_Month_Offset <= 0 and \
                Snp_Seq_No == 6"

d = pd.date_range(start_date, today + pd.offsets.MonthEnd(n_test_periods), freq="m")
eom = pd.DataFrame(d, columns=["End_of_Month"])

# eom = pipehist_pivot1[['End_of_Month']].drop_duplicates(subset='End_of_Month',keep='last').sort_values(by=['End_of_Month']).reset_index(drop=True)

sub_region = pd.DataFrame(
    pipehist1["Sub_Region_Code"].unique(), columns=["Sub_Region_Code"]
)
snp_date = pd.DataFrame(pipehist1["Snapshot_Date"].unique(), columns=["Snapshot_Date"])

eom1 = df_crossjoin(snp_date, eom)
eom1 = eom1.reset_index(drop=True)

eom2 = df_crossjoin(eom1, sub_region)
eom2 = eom2.reset_index(drop=True)
# eom1 = eom.merge(snp_date, how="cross")
# eom2 = eom1.merge(sub_region, how="cross")


eom_region_snp = eom2.replace(np.nan, "NA", regex=True).query(
    "Sub_Region_Code not in @exclude_sub_region"
)
eom_region_snp = eom_region_snp.reset_index(drop=True)
# eom_region_snp.drop(columns=['key'], axis=1, inplace = True)

# Add offset columns

eom_region_snp["Relative_Month_Offset"] = round(
    (eom_region_snp["End_of_Month"] - current_eom) / np.timedelta64(1, "M"), 0
).astype(int)
eom_region_snp["Relative_Snapshot_Month_Offset"] = round(
    (eom_region_snp["Snapshot_Date"] + pd.offsets.MonthEnd(0) - current_eom)
    / np.timedelta64(1, "M"),
    0,
).astype(int)
eom_region_snp["Relative_Offset"] = (
    eom_region_snp["Relative_Month_Offset"] -
    eom_region_snp["Relative_Snapshot_Month_Offset"]
)
eom_region_snp["Relative_Offset"] = eom_region_snp["Relative_Offset"].astype(int)
eom_region_snp["Snapshot_Day_of_Month"] = eom_region_snp["Snapshot_Date"].dt.day

eom_region_snp1 = eom_region_snp.copy()

# display(eom_region_snp.tail())

# Add a sequence number to each element in a group using python
seq = pd.DataFrame()
seq["Snp_Seq_No"] = [0, 1, 2, 3, 4, 5, 6]

eom_region_snp1 = eom_region_snp1.sort_values(by=["End_of_Month", "Snapshot_Date", "Sub_Region_Code"]).reset_index(drop=True)
#print("eom_region_snp1:")
#eom_region_snp1.info()
#display(eom_region_snp1.tail())

eom_region_snp2 = (
    eom_region_snp1[["End_of_Month", "Relative_Offset", "Snapshot_Date"]]
    .groupby(["End_of_Month", "Relative_Offset", "Snapshot_Date"])
    .last()
)

# eom_region_snp['Snp_Seq_No'] = normalize(eom_region_snp['Snapshot_Date'].dt.day,1,15) #Normalizing from one range to another

eom_region_snp2["Snp_Seq_No"] = eom_region_snp2.groupby(
    ["End_of_Month", "Relative_Offset"]
).cumcount()

cols1 = ["End_of_Month", "Relative_Offset"]
eom_offset_region = eom_region_snp1[cols1].drop_duplicates()
eom_offset_seq = df_crossjoin(eom_offset_region, seq)

#display(eom_offset_seq)
eom_region_snp2 = eom_region_snp2.reset_index(
    level=eom_region_snp2.index.names
).reset_index(drop=True)

#print("eom_region_snp2:")
#eom_region_snp2.info()
#display(eom_region_snp2)
#display(seq.info())

eom_region_snp3 = pd.merge(
    eom_offset_seq,
    eom_region_snp2,
    how = "left",
    on=["End_of_Month","Relative_Offset","Snp_Seq_No"]
)
eom_region_snp3 = eom_region_snp3.reset_index(drop=True)
#print("eom_region_snp3:")
#display(eom_region_snp3.info())

#display(eom_region_snp3.tail())

eom_region_snp3 = eom_region_snp3.ffill(axis=0)

eom_region_snp_final = pd.merge(
    eom_region_snp1,
    eom_region_snp3,
    how="right",
    on=["End_of_Month", "Relative_Offset", "Snapshot_Date"],
).fillna(nan_value)
eom_region_snp_final = eom_region_snp_final.reset_index(drop=True)

eom_region_snp_final = eom_region_snp_final.replace(np.nan, 0, regex=True)
#print("eom_region_snp_final:")
#display(eom_region_snp_final.info())

int_cols = [
    "Relative_Snapshot_Month_Offset",
    "Relative_Month_Offset",
    "Relative_Offset",
    "Snp_Seq_No",
    "Snapshot_Day_of_Month"
]

eom_region_snp_final = coerce_to_int(eom_region_snp_final)


# FILTER for relevant history and forecast period rows
eom_region_snp_final = eom_region_snp_final.query(main_filter).sort_values(by=sort_cols_snp).reset_index(drop=True)

eom_region_snp_final.to_csv(output_data_path + "eom_region_snp.csv", index=False)

print("eom_region_snp_final:")
#eom_region_snp_final['End_of_Month'].unique()
eom_region_snp_final.info()
eom_region_snp_final.tail()


In [ ]:
pd.DataFrame(eom_region_snp_final.describe(include='all').transpose().head())
#'''
eom_region_snp_pvt = eom_region_snp_final.pivot_table(
    index=[time_column_name, "Relative_Offset"],
    columns=["Snp_Seq_No", "Sub_Region_Code"],
    values=["Snapshot_Date"],
    aggfunc={"Snapshot_Date": "count"},
    margins=False,
)
# eom_region_snp_pvt = eom_region_snp_pvt.reset_index(level=eom_region_snp_pvt.index.names).reset_index(drop=True)
eom_region_snp_pvt
#'''

In [ ]:

# Pivot Talent from long to wide to match month end grain
print("Preparing talent wide")
nan_value = 0
#display(talenthist1.tail())

talenthist_wide1 = talenthist1.pivot_table(
    index=["End_of_Month", "Sub_Region_Code"],
    columns=["Billable", "Journey_Level"],
    values=["Headcount"],
    aggfunc={"Headcount": np.sum},
    margins=False,
).fillna(nan_value)


talenthist_wide1.columns = [
    "_".join(tuple(map(str, t))) for t in talenthist_wide1.columns.values
]

talenthist_wide1.reset_index(inplace=True)
talent_cols1 = talenthist_wide1.columns
remove_list1 = ["End_of_Month", "Sub_Region_Code", "Headcount"]
talent_cols1 = difflist(talent_cols1, remove_list1)
#print(talent_cols1)

# Sum contingent at month, billable, sub-region level
talenthist_wide2 = talenthist1.pivot_table(
    index=["End_of_Month", "Sub_Region_Code"],
    columns=["Billable"],
    values=["Headcount_Contingent"],
    aggfunc={"Headcount_Contingent": np.sum},
    margins=False,
).fillna(nan_value)

talenthist_wide2.columns = [
    "_".join(tuple(map(str, t))) for t in talenthist_wide2.columns.values
]

talenthist_wide2.reset_index(inplace=True)
talent_cols2 = talenthist_wide2.columns
remove_list2 = ["End_of_Month", "Sub_Region_Code", "Headcount_Contingent"]
talent_cols2 = difflist(talent_cols2, remove_list2)
#print(talent_cols2)

# talenthist_wide.tail()
# pipetrend_wide['Pipeline'] = pd.to_numeric(pipetrend_wide['Pipeline'], errors='coerce').astype(int)

talent_cols = talent_cols1 + talent_cols2
numeric_cols_talent = numeric_cols
numeric_cols_talent = numeric_cols_talent + talent_cols
numeric_cols = numeric_cols_talent
print(numeric_cols)

talenthist_wide1 = coerce_to_numeric(talenthist_wide1)
talenthist_wide1 = convert_date_cols(talenthist_wide1)
talenthist_wide1 = talenthist_wide1.sort_values(by=sort_cols_eom).reset_index(drop=True)

talenthist_wide2 = coerce_to_numeric(talenthist_wide2)
talenthist_wide2 = convert_date_cols(talenthist_wide2)
talenthist_wide2 = talenthist_wide2.sort_values(by=sort_cols_eom).reset_index(drop=True)

# print(' pipetrend_wide: ')

#display(talenthist_wide1.info())
#display(talenthist_wide2.info())
'''
# pipetrend_wide.query('Fin_Entity_ID==@entity_debug')
#print("talenthist_wide:", talenthist_wide.shape)
#talenthist_wide.tail()  # .query('Relative_Snapshot_Month_Offset == 0')
'''

In [ ]:

# Pivot ProjectActuals from long to wide to match month end grain
print("Preparing ProjectActuals wide")
nan_value = 0

projectactualshist_wide = projectactualshist1.pivot_table(
    index=["End_of_Month", "Sub_Region_Code"],
    columns="Journey_Level",
    values=["Utilization_Billable"],
    aggfunc={"Utilization_Billable": np.sum},
    margins=False,
).fillna(nan_value)


projectactualshist_wide.columns = [
    "_".join(tuple(map(str, t))) for t in projectactualshist_wide.columns.values
]

projectactualshist_wide.reset_index(inplace=True)
projectactuals_cols = projectactualshist_wide.columns
remove_list = ["End_of_Month", "Sub_Region_Code", "Utilization_Billable"]
projectactuals_cols = difflist(projectactuals_cols, remove_list)
#print(projectactuals_cols)
    
numeric_cols_projectactuals = numeric_cols
numeric_cols_projectactuals = numeric_cols_projectactuals + projectactuals_cols
print(numeric_cols_projectactuals)

numeric_cols = numeric_cols_projectactuals
projectactualshist_wide = coerce_to_numeric(projectactualshist_wide)
projectactualshist_wide = convert_date_cols(projectactualshist_wide)
projectactualshist_wide = projectactualshist_wide.sort_values(by=sort_cols_eom).reset_index(drop=True)

# print(' projectactualshist_wide: ')

display(projectactualshist_wide.info())
#display(projectactualshist_wide.tail())
'''
# projectactualshist_wide.query('Fin_Entity_ID==@entity_debug')
print("projectactualshist_wide:", projectactualshist_wide.shape)
'''


In [ ]:

# Pivot Pipeline_Trend from long to wide to match month end grain
'''
print("Preparing Pipeline_Trend Wide")

nan_value = 0
pipetrend_wide = pipetrend1.pivot_table(
    index=["Snapshot_Date", "End_of_Month", "Sub_Region_Code"],
    columns="Pipeline_Type",
    values=["Pipeline_Trend"],  # , 'Pipeline_3M_Rolling_Avg'
    aggfunc={"Pipeline_Trend": np.sum},
    margins=False,
).fillna(nan_value)


pipetrend_wide.columns = [
    "_".join(tuple(map(str, t))) for t in pipetrend_wide.columns.values
]

pipe_cols = [
    "Active_Unrecognized_Trend",
    "Opportunity_Trend",
    "Opportunity_ML_Trend",  # ,    "Recognized",
]

pipetrend_wide.columns = pipe_cols

pipetrend_wide = pipetrend_wide.reset_index(
    level=pipetrend_wide.index.names
).reset_index(drop=True)

numeric_cols_pipetrend = numeric_cols
numeric_cols_pipetrend = numeric_cols_pipetrend + pipe_cols
if "Pipeline_Trend" in numeric_cols_pipetrend:
    numeric_cols_pipetrend.remove("Pipeline_Trend")
print(numeric_cols_pipetrend)

numeric_cols = numeric_cols_pipetrend
pipetrend_wide = convert_date_cols(pipetrend_wide)
pipetrend_wide = coerce_to_numeric(pipetrend_wide)
pipetrend_wide = coerce_to_int(pipetrend_wide)

pipetrend_wide = pipetrend_wide.sort_values(by=["Snapshot_Date", "End_of_Month", "Sub_Region_Code"]).reset_index(drop=True)

# print(' pipetrend_wide: ')

display(pipetrend_wide.info())

# pipetrend_wide.query('Fin_Entity_ID==@entity_debug')
print("pipetrend_wide:", pipetrend_wide.shape)
#pipetrend_wide.tail()  # .query('Relative_Snapshot_Month_Offset == 0')

'''

In [ ]:

# Merge eom_studio with Revenue, Talent, Opportunity, Project
print("Preparing merge_df")
from functools import reduce

nan_value = 0
merge_dfs = [eom_region_snp_final, revhist1, talenthist_wide1, talenthist_wide2, opphist1, projhist1, projectactualshist_wide]

merge_df = reduce(
    lambda left, right: pd.merge(
        left, right, how="left", on=["End_of_Month", "Sub_Region_Code"]
    ),
    merge_dfs,
).fillna(nan_value)


merge_df = convert_date_cols(merge_df)
merge_df = coerce_to_numeric(merge_df)
merge_df = coerce_to_int(merge_df)

merge_df = merge_df.replace(np.nan, 0, regex=True)
merge_df.sort_values(by=sort_cols_snp).reset_index(drop=True)

# merge_df.drop(columns=['Current_Opp._Period_Value','Conversions'], axis=1, inplace = True)

#print(" merge_df: ", merge_df.shape)

# merge_df.to_csv(output_data_path   'merge_df.csv', index=False)
# show_stats(merge_df)
merge_df.info()
#print(merge_df.shape)
#merge_df.tail()  # .query('Fin_Entity_ID==@entity_debug')')



In [ ]:

# Merge with eom_region_snp1 with stock_final_pivot
print("Preparing merge_df1")
# eom1 = eom  # pd.DataFrame(d, columns=['End_of_Month'])
# stock_final_df = pd.merge(
#    eom_region_snp1, stock_final_pivot, how="left", on=["End_of_Month","Relative_Month_Offset","Relative_Snapshot_Month_Offset","Relative_Offset"]
# ).fillna(nan_value)

# sp500_df = sp500_df.ffill(axis = 0)

# stock_final_df = stock_final_df.sort_values(by=sort_cols_eom).reset_index(drop=True)
# display(sp500_df.tail(10))
merge_df1 = pd.merge(merge_df, stock_final_pivot, how="left", on="End_of_Month").fillna(
    nan_value
)
print(symbols_name)

numeric_cols_stock = numeric_cols
numeric_cols_stock = numeric_cols_stock + symbols_name
#if "Pipeline_Trend" in numeric_cols_stock:
#    numeric_cols_stock.remove("Pipeline_Trend")
print(numeric_cols_stock)

numeric_cols = numeric_cols_stock
merge_df1 = convert_date_cols(merge_df1)
merge_df1 = coerce_to_numeric(merge_df1)
merge_df1 = coerce_to_int(merge_df1)

merge_df1 = merge_df1.sort_values(by=sort_cols_snp).reset_index(drop=True)
# merge_df1.to_csv(output_data_path   'merge_df1.csv', index=False)
# merge_sdf1 = spark.createDataFrame(merge_df1);
# merge_sdf1.write.format("parquet").mode("overwrite").parquet(output_data_path   'merge_df1.parquet')

# show_stats(merge_df1.tail(10))
# merge_df1.info()
#print(merge_df1.shape)
merge_df1.info()
#merge_df1.tail()



In [ ]:

# Pivot Pipeline_Type from long to wide to match month end grain
print("Preparing pipehist_wide")
pipehist_long = pipehist1.copy()
pipehist_wide = pipehist_long.pivot_table(
    index=[
        "Snapshot_Date",
        "Relative_Snapshot_Month_Offset",
        "End_of_Month",
        "Relative_Month_Offset",
        "Sub_Region_Code"
    ],
    columns="Pipeline_Type",
    values=["Pipeline", "Pipeline_at_100_Percent", "Yield"],  # , 'Pipeline_3M_Rolling_Avg'
    # aggfunc={"Pipeline": np.sum},
    margins=False,
).fillna(nan_value)

pipehist_wide.columns = [
    "_".join(tuple(map(str, t))) for t in pipehist_wide.columns.values
]

pipehist_wide.reset_index(inplace=True)
pipe_cols = pipehist_wide.columns
remove_list = ["Pipeline", "Pipeline_at_100_Percent", "Yield"]
pipe_cols = difflist(projectactuals_cols, remove_list)
#print(pipe_cols)
    
numeric_cols_pipe = numeric_cols
numeric_cols_pipe = numeric_cols_pipe + pipe_cols
print(numeric_cols_pipe)

numeric_cols = numeric_cols_pipe
pipehist_wide = convert_date_cols(pipehist_wide)
pipehist_wide = coerce_to_numeric(pipehist_wide)
pipehist_wide = coerce_to_int(pipehist_wide)

pipehist_wide = pipehist_wide.sort_values(by=["End_of_Month", "Snapshot_Date", "Sub_Region_Code"]).reset_index(drop=True)

# print(' pipehist_wide: ')

# pipehist_wide.query('Fin_Entity_ID==@entity_debug')
print("Pipehist_wide:", pipehist_wide.shape)

pd.DataFrame(pipehist_wide.describe(include='all').transpose().head())

#pipehist_wide.tail()  # .query('Relative_Snapshot_Month_Offset == 0')



In [ ]:

# FINAL MERGE
print("Preparing merge_final")
pipehist_wide1 = pipehist_wide.copy()
merge_df2 = pd.merge(
    pipehist_wide1,
    pipetrend1,
    how="left",
    on=["End_of_Month", "Snapshot_Date", "Sub_Region_Code"],
).fillna(nan_value)

display(merge_df2.tail())

merge_final = pd.merge(
    merge_df1,
    merge_df2,
    how="left",
    on=[
        "End_of_Month",
        "Relative_Month_Offset",
        "Snapshot_Date",
        "Relative_Snapshot_Month_Offset",
        "Sub_Region_Code",
    ],
).fillna(nan_value)
# print(merge_final.info())
# print(merge_final.tail())
# merge_final.drop(columns=['Snapshot_Date_Short','Relative_Snapshot_Month_Offset'], axis=1, inplace = True)
# New Code end
#display(merge_final.tail())
merge_final = merge_final.infer_objects()
merge_final = convert_date_cols(merge_final)
merge_final = coerce_to_numeric(merge_final)
merge_final = coerce_to_int(merge_final)

merge_final = merge_final.sort_values(by=sort_cols_snp).reset_index(drop=True)


merge_final.to_csv(output_data_path + "merge_final.csv", index=False)
merge_final.to_parquet(output_data_path + "merge_final.parquet", index=None)
"""
#merge_final_sdf = spark.createDataFrame(merge_final);
#merge_final_sdf.repartition(1).write.format("parquet").mode("overwrite").parquet(output_data_path + 'merge_final.parquet')
"""
print("merge_final:", merge_final.shape)
# merge_final.tail()
#print(merge_final["Snapshot_Date"].unique())
# merge_final.query('Relative_Snapshot_Month_Offset == 0')
# show_stats(merge_final)



In [ ]:
merge_final.query('Relative_Offset == 0 and Snp_Seq_No == 6').tail()

In [ ]:
pd.DataFrame(merge_final.describe(include='all').transpose().head())